# Notebook V1.0 Base model - 67% accuracy
notebook is hosted on: https://colab.research.google.com/drive/1zI9G_oxc3MgtywFGmJHDnZcZYK416JXO#scrollTo=vVUQNfll3MwR

**The first two cells are used to mount the data when the google colab session expires. Please ignore**

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip /content/drive/MyDrive/dog_breeds/dog-breed-identification.zip

# IMPORTS

In [ ]:
import gc

# for warnings
import warnings
warnings.filterwarnings("ignore")

# utility libraries
import os
import copy
import tqdm
import numpy as np 
import pandas as pd 
import cv2, random, time, shutil, csv
import tensorflow as tf
import math

import tensorflow.keras as K
import tensorflow as tf

# keras libraries
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from keras.utils import to_categorical
from keras import backend as kb

# sklearn libs
from sklearn.model_selection import train_test_split

## READING IMAGES
**Narrowing down the training data to just 10 breeds as required by the assignment**

In [ ]:
# Note: Training for all 120 breeds should yield better results (planned for Notebook V2)
BREEDS = ["beagle", "chihuahua", "doberman","french_bulldog", "golden_retriever", "malamute", "pug", "saint_bernard", "scottish_deerhound", "tibetan_mastif"]

data_dir = '/content/'
data_df = pd.read_csv(os.path.join(data_dir, 'labels.csv'))
data_df = data_df[data_df.breed.isin(BREEDS)]

# Note: No images with label "tibetan_mastif" were found in the training set, hence only 9 classes were found after filtering
class_names = sorted(data_df['breed'].unique())
images_list = sorted(os.listdir(os.path.join(data_dir, 'train')))
to_size = (244, 244)

X = []
Y = []

for image in tqdm.tqdm(images_list):
    try:
      cls_name = data_df[data_df['id'] == image[:-4]].iloc[0,1]
    except IndexError:
      continue
    cls_index  = int(class_names.index(cls_name)) 
    image_path = os.path.join(data_dir, 'train',image)
    orig_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    res_image  = cv2.resize(orig_image, to_size)

    X.append(res_image)
    Y.append(cls_index)

No. of classes read - 9


100%|██████████| 10222/10222 [00:11<00:00, 901.34it/s]


# Converting to arrays

In [ ]:
print(len(X), len(Y))
Xarr = np.array(X)
Yarr = np.array(Y).reshape(-1,1)

del(X)
print(Xarr.shape, Yarr.shape)
gc.collect()

772 772
(772, 244, 244, 3) (772, 1)


4065

# Preprocessing function
**Notebook V1.2 will augment images in this function and increase training data**

In [ ]:
def preprocess_data(X, Y):
  X_p = K.applications.resnet50.preprocess_input(X)
  Y_p = K.utils.to_categorical(Y, 10)
  return X_p, Y_p

# Splitting train and test data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Xarr, Yarr, test_size=0.33, random_state=42)
print(x_train.shape, y_train.shape)

x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)
print(x_train.shape, y_train.shape)

(517, 244, 244, 3) (517, 1)
(517, 244, 244, 3) (517, 10)


# Preparing Resnet model

**Setting input layer and freezing first 142 layers as per imagenet weights**

In [ ]:
input_tensor = K.Input(shape=(244, 244, 3))
res_model = K.applications.ResNet50(include_top=False, weights="imagenet", input_tensor=input_tensor)

for layer in res_model.layers[:143]:
  layer.trainable = False

In [ ]:
model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(256, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(10, activation='softmax'))

# Training model

In [ ]:
check_point = K.callbacks.ModelCheckpoint(filepath='dogbreeds.h5', monitor='val_accuracy', mode='max', save_best_only=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=K.optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_test, y_test), callbacks=[check_point])
model.summary()

Epoch 1/10
17/17 [==============================] - 221s 13s/step - loss: 2.9853 - accuracy: 0.1052 - val_loss: 2.6030 - val_accuracy: 0.1294
Epoch 2/10
17/17 [==============================] - 209s 12s/step - loss: 2.7507 - accuracy: 0.1217 - val_loss: 2.2062 - val_accuracy: 0.1961
Epoch 3/10
17/17 [==============================] - 210s 12s/step - loss: 2.6883 - accuracy: 0.1359 - val_loss: 2.0243 - val_accuracy: 0.2706
Epoch 4/10
17/17 [==============================] - 215s 13s/step - loss: 2.4606 - accuracy: 0.1825 - val_loss: 1.8790 - val_accuracy: 0.4000
Epoch 5/10
17/17 [==============================] - 210s 12s/step - loss: 2.3472 - accuracy: 0.1982 - val_loss: 1.8325 - val_accuracy: 0.4941
Epoch 6/10
17/17 [==============================] - 210s 13s/step - loss: 2.4637 - accuracy: 0.1646 - val_loss: 1.7738 - val_accuracy: 0.5451
Epoch 7/10
17/17 [==============================] - 216s 13s/step - loss: 2.3263 - accuracy: 0.2302 - val_loss: 1.7268 - val_accuracy: 0.6078
Epoch 